In [ ]:
import yaml
import pathlib
import pandas as pd

In [ ]:
template_dir = pathlib.Path().absolute()

In [ ]:
snakemake = f"""
template_dir = '{template_dir}'
template_dir = template_dir.rstrip('/')

rule all:
    input:
        'finish'

"""

nbs = pd.Series({
    p.name.split('-')[0]: p.name.split('.')[0]
    for p in template_dir.glob('*-*.ipynb')
}).sort_index()
for rule_id, rule_nb_name in nbs.items():
    if rule_id == nbs.index[-1]:
        last_flag = 'flag="finish"'
    else:
        last_flag = ''
        
    rule_text = f"""
rule r{rule_id}:
    input:
        nb=f"{{template_dir}}/{rule_nb_name}.ipynb",
        config=f'config/{rule_id}.yaml',
    output:
        {last_flag}
    log:
        nb="{rule_nb_name}.ipynb",
        log="log/{rule_nb_name}.log"
    threads: 1
    shell:
        "papermill -f {{input.config}} "
        "{{input.nb}} {{log.nb}} "
        "> {{log.log}} 2>&1"
"""
    snakemake += rule_text
with open('../L1/Snakefile_template', 'w') as f:
    f.write(snakemake)